Imports


In [1]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import datetime
import time
import keras.backend as K
import math

from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Bidirectional, Dropout, TimeDistributed
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import auc, roc_curve, auc, mean_squared_error, mean_absolute_error
from matplotlib import pyplot as plt
from numpy import concatenate
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
# import tf.keras.preprocessing.timeseries_dataset_from_array

Paths y lectura de datos

In [2]:
full_data_path = "/content/drive/MyDrive/datos_web/otros/test3.csv"
ordered_data_path = "/content/drive/MyDrive/datos_web/otros/cleaned_ordered.csv"
# ordered_data_path = "/content/drive/MyDrive/datos_web/otros/cleaned_ordered_2.csv"
final_data_path = "/content/drive/MyDrive/datos_web/otros/final_data.csv"

# test_cleaned_full_path = "/content/drive/MyDrive/datos_web/otros/full_cleaned.csv"

In [4]:
final_data = pd.read_csv(final_data_path)
dates = pd.to_datetime(final_data["Unnamed: 0"].values)
final_data.drop("Unnamed: 0", axis = 1, inplace=True)
final_data.set_index(dates, inplace=True)

# final_data.set_
ordered_data = pd.read_csv(ordered_data_path)
ordered_data.drop("Unnamed: 0", axis = 1, inplace=True)

Variables y grupos. Seleccionar datos del conjunto. Sirve para análisis



In [ ]:
barrios = {}
tipos = {}
carriles = {}
entradas = {
    "serreria" : ["A233", "A415", "A235"],
    "avcid" : ["A143", "A170", "A71", "A72"],
    "blasco" : ["A49", "A65", "A33", "A47"]
}

print(entradas)
salidas = {
    "serreria" : ["A74", "A5", "A51"],
    "avcid" : ["A72", "A245"],
    "blasco" : ["A234", "A52"]
}
print(salidas)

avcid = ["A145", "A70", "A71", "A72", "A69"]
# alameda = ["A174", "A345" ,"A266", "A277", "A166", "A173"]
alameda = ["A174", "A345" ,"A173"]
# genaro = ["B24", "B118", "B81"]
genaro = ["B24", "B81", "B118", "B57"]
# avmed = ["A131", "A179", "A104", "A88", "A351" ]
avmed = ["A131", "A179", "A351" ]
# justopastor = ["A415", "A359", "A92", "A230","A231"]
justopastor = ["A415", "A359", "A92", "A231"]
# baron = ["A98", "B29", "B10"]
baron = ["A98", "B10"]
# Barrios
grouped = ordered_data.groupby(ordered_data.barrio)
for name, group in grouped:
  neighs = list(set(group.ATA))
  barrios[name] = neighs
print(barrios)

# Tipos de calle
grouped = ordered_data.groupby(ordered_data.tipo)
for name, group in grouped:
  types = list(set(group.ATA))
  tipos[name] = types
print(tipos)

# Mismos carriles
grouped = ordered_data.groupby(ordered_data.carriles)
for name, group in grouped:
  lanes = list(set(group.ATA))
  carriles[name] = lanes
print(carriles)

In [6]:
barrio = barrios['algiros']

datos = final_data[barrio]
# datos = datos.iloc[datos.index.dayofweek == 0]
# Para entrenar modelo final
train = datos["2021-03-15 00:00:00":"2021-06-06 23:00:00"]
# test = datos["2021-06-07 00:00:00":"2021-06-20 23:00:00"]

# train = datos["2021-03-15 00:00:00":"2021-05-16 23:00:00"]
# val = datos["2021-05-17 00:00:00":"2021-06-06 23:00:00"]
test = datos["2021-06-07 00:00:00":"2021-06-20 23:00:00"]


mondays_train = train.iloc[train.index.dayofweek == 0]
tuesdays_train = train.iloc[train.index.dayofweek == 1]
wednesdays_train = train.iloc[train.index.dayofweek == 2]
thursdays_train = train.iloc[train.index.dayofweek == 3]
fridays_train = train.iloc[train.index.dayofweek == 4]
saturdays_train = train.iloc[train.index.dayofweek == 5]
sundays_train = train.iloc[train.index.dayofweek == 6]

mondays_test = test.iloc[test.index.dayofweek == 0]
tuesdays_test = test.iloc[test.index.dayofweek == 1]
wednesdays_test = test.iloc[test.index.dayofweek == 2]
thursdays_test = test.iloc[test.index.dayofweek == 3]
fridays_test = test.iloc[test.index.dayofweek == 4]
saturdays_test = test.iloc[test.index.dayofweek == 5]
sundays_test = test.iloc[test.index.dayofweek == 6]

features = datos.shape[1]

In [7]:
barr = list(set(barrios.keys()))

Para crear la serie temporal completa, esta parte

In [15]:

datos = final_data

datos = datos["2021-03-15 00:00:00":"2021-06-20 23:00:00"]
datos_pre = datos["2021-03-15 00:00:00":"2021-05-09 23:00:00"]
datos_post = datos["2021-05-10 00:00:00":"2021-06-20 23:00:00"]
datos = datos["2021-03-15 00:00:00":"2021-06-20 23:00:00"]
datos.fillna(0, inplace=True)

# datos = datos.iloc[datos.index.dayofweek == 0]
# Para entrenar modelo final
train = datos["2021-03-15 00:00:00":"2021-06-06 23:00:00"]
test = datos["2021-06-07 00:00:00":"2021-06-20 23:00:00"]

# train = datos["2021-03-15 00:00:00":"2021-05-16 23:00:00"]
val = datos["2021-05-17 00:00:00":"2021-06-06 23:00:00"]
# test = datos["2021-06-07 00:00:00":"2021-06-20 23:00:00"]


mondays_train = train.iloc[train.index.dayofweek == 0]
tuesdays_train = train.iloc[train.index.dayofweek == 1]
wednesdays_train = train.iloc[train.index.dayofweek == 2]
thursdays_train = train.iloc[train.index.dayofweek == 3]
fridays_train = train.iloc[train.index.dayofweek == 4]
saturdays_train = train.iloc[train.index.dayofweek == 5]
sundays_train = train.iloc[train.index.dayofweek == 6]
weekdays_train = mondays_train.append(tuesdays_train).append(wednesdays_train).append(thursdays_train).append(fridays_train)
weekends_train = saturdays_train.append(sundays_train)

mondays_test = test.iloc[test.index.dayofweek == 0]
tuesdays_test = test.iloc[test.index.dayofweek == 1]
wednesdays_test = test.iloc[test.index.dayofweek == 2]
thursdays_test = test.iloc[test.index.dayofweek == 3]
fridays_test = test.iloc[test.index.dayofweek == 4]
saturdays_test = test.iloc[test.index.dayofweek == 5]
sundays_test = test.iloc[test.index.dayofweek == 6]
weekdays_test = mondays_test.append(tuesdays_test).append(wednesdays_test).append(thursdays_test).append(fridays_test)
weekends_test = saturdays_test.append(sundays_test)
features = datos.shape[1]

Crear la serie temporal.

2 formas diferentes :
  1 - Método create_dataset
  2 - Método series_to_supervised
  

In [ ]:
# # Convertir la serie temporal en datos supervisados
# Serie sin tener en cuenta informacion adicional, solo la hora anterior
def series_to_supervised(data, keys, n_in=1, n_out=1, dropnan=True):
  n_vars = 1 if type(data) is list else data.shape[1]
  df = pd.DataFrame(data)
  cols, names = list(), list()
  # input sequence (t-n, ... t-1)
  for i in range(n_in, 0, -1):
    cols.append(df.shift(i))
    names += [('%s(t-%d)' % (keys[j], i)) for j in range(n_vars)]
  # forecast sequence (t, t+1, ... t+n)
  for i in range(0, n_out):
    cols.append(df.shift(-i))
    if i == 0:
      names += [('%s(t)' % (keys[j])) for j in range(n_vars)]
    else:
      names += [('%s(t+%d)' % (keys[j], i)) for j in range(n_vars)]
  # put it all together
  agg = pd.concat(cols, axis=1)
  agg.columns = names
  # drop rows with NaN values
  if dropnan:
    agg.dropna(0, inplace=True)
  return agg


features = train.shape[1]
timesteps = 2
outputs = 1

## Cambiar en funcion de lo que se quiera
# train = weekdays_train
# test = weekdays_test
# val = weekends_val
# train = weekends_train
# # test = weekends_test

features = train.shape[1]

train_keys = train.keys()
test_keys = test.keys()
val_keys = val.keys()
print(test.iloc[0:24])
scaler = MinMaxScaler(feature_range=(0,1))
scaler = scaler.fit(train)
train_values = scaler.transform(train)
test_values = scaler.transform(test)
val_values = scaler.transform(val)

serie_train = series_to_supervised(train_values, train_keys, n_in=1, n_out=2)
serie_test = series_to_supervised(test_values, test_keys,  n_in=1, n_out=2)
serie_val = series_to_supervised(val_values, val_keys, n_in=1, n_out=2)

x_train, y_train = serie_train.values[:, :-features], serie_train.values[:, -features:]
x_test, y_test = serie_test.values[:, :-features], serie_test.values[:, -features:]
x_val, y_val = serie_val.values[:, :-features], serie_val.values[:, -features:]

##############  Resize de los arrays ##########
x_train = np.asarray(x_train).astype('float32')
x_test = np.asarray(x_test).astype('float32')
x_val = np.asarray(x_val).astype('float32')

y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')
y_val = np.asarray(y_val).astype('float32')

x_train = np.reshape(x_train, (x_train.shape[0], timesteps, features))
x_test = np.reshape(x_test, (x_test.shape[0], timesteps, features))
x_val = np.reshape(x_val, (x_val.shape[0], timesteps, features))


Creación del modelo

In [ ]:
####### Crear y entrenar el modelo  ##########
batch_size = 24
epochs = 200
# print(features, "FEATURES")
# print(x_train.shape[0], "MUESTRAS")
# print(timesteps, "TIMESTEPS")

model = Sequential()
model.add(LSTM(150, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2]), name="Input"))
model.add(Dropout(0.2, name= "Dropout_1"))
model.add(Dense(units = features, name = "Output", activation='linear'))
model.summary()
metrics=["mse", "mae"]
model.compile(metrics=metrics, loss='mae', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)) 

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [20]:
filepath="weights.best.hdf5"
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
checkpoint = ModelCheckpoint(filepath, monitor='val_mae', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint, es]

In [ ]:
# Para ajustar
history = model.fit(
    x_train, y_train, 
    epochs=epochs, 
    batch_size=batch_size, 
    validation_data=(x_val, y_val),
    callbacks=callbacks_list,
    shuffle=False,
    verbose=1)

In [ ]:
# Final
history = model.fit(
    x_train, y_train, 
    epochs=epochs, 
    batch_size=batch_size, 
    validation_data=(x_test, y_test),
    callbacks=callbacks_list,
    shuffle=False,
    verbose=1)

In [ ]:
plt.title("MAE")
plt.xlabel("Épocas")
plt.ylabel("MAE")
plt.plot(history.history['mae'], label='entrenamiento')
plt.plot(history.history['val_mae'], label='test')
plt.legend()
plt.show()


plt.title("MSE")
plt.xlabel("Épocas")
plt.ylabel("MSE")
plt.plot(history.history['mse'], label='entrenamiento')
plt.plot(history.history['val_mse'], label='test')
plt.legend()
plt.show()

In [ ]:
val_rmse_values = []
train_rmse_values = []
for i in history.history['val_mse']:
  val_rmse_values.append(math.sqrt(i))
for i in history.history['mse']:
  train_rmse_values.append(math.sqrt(i))


plt.title("RMSE")
plt.plot(train_rmse_values, label='entrenamiento')
plt.plot(val_rmse_values, label='test')
plt.legend()
plt.xlabel("Épocas")
plt.ylabel("RMSE")
plt.show()

In [ ]:
model.load_weights("/content/weights.best.hdf5")
# model.load_weights("/content/drive/MyDrive/datos_web/otros/weights.best.hdf5")
# y_test = y_train
# # make predictions
train_predict = model.predict(x_train)
test_predict = model.predict(x_test)
val_predict = model.predict(x_val)
print(train_predict)
train_predict_inversed = scaler.inverse_transform(train_predict)
test_predict_inversed = scaler.inverse_transform(test_predict)  
val_predict_inversed = scaler.inverse_transform(val_predict)  
y_test_inversed = scaler.inverse_transform(y_test)  
y_train_inversed = scaler.inverse_transform(y_train)
y_val_inversed = scaler.inverse_transform(y_val)


total_train_rmse = 0
total_train_mae = 0
total_train_mse = 0

total_test_rmse = 0
total_test_mae = 0
total_test_mse = 0
for i in range(test_predict.shape[1]):
  train_score = math.sqrt(mean_squared_error(y_train[:, i], train_predict[:,  i]))
  train_mse_score = mean_squared_error(y_train[:, i], train_predict[:,  i])
  train_mae_score = mean_absolute_error(y_train[:, i], train_predict[:,  i])
  print('Train Score: %.5f RMSE' % (train_score))
  print('Train Score: %.5f MAE' % (train_mae_score))
  print('Train Score: %.5f MSE' % (train_mse_score))
  total_train_rmse += train_score
  total_train_mae += train_mae_score
  total_train_mse += train_mse_score
  # # print(len(test_predict))
  # # print(len(y_test))
  test_score = math.sqrt(mean_squared_error(y_test[:, i], test_predict[:,  i]))
  test_mae_score = mean_absolute_error(y_test[:, i], test_predict[:,  i])
  test_mse_score = mean_squared_error(y_test[:, i], test_predict[:,  i])
  print('Test Score: %.5f RMSE' % (test_score))
  print('Test Score: %.5f MAE' % (test_mae_score))
  print('Test Score: %.5f MSE' % (test_mse_score))
  total_test_rmse += test_score
  total_test_mae += test_mae_score
  total_test_mse += test_mse_score
  fig, ax = plt.subplots()
  predicted_values = test_predict_inversed[:, i]
  real_values = y_test_inversed[:, i]
  # print(predicted_values)
  horas = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
           20, 21, 22, 23]
  ax.plot(horas, real_values[22:46], color = 'red', label = 'Reales')
  ax.plot(horas, predicted_values[22:46], color = 'blue', label = 'Predichos')
  # ax.plot(real_values, color = 'red', label = 'Reales')
  # ax.plot(predicted_values, color = 'blue', label = 'Predichos')
  # ax.set_title(train_keys[i])
  if len(ordered_data[ordered_data.ATA==train_keys[i]]) > 0:
    name = ordered_data[ordered_data.ATA==train_keys[i]].desc.iloc[0]
  else:
    name = train_keys[i]
  # if len(name) > 15:
    # name = name[0:20] +"..."
  ax.set_xlabel('Hora')
  ax.set_title(name)
  # ax.set_xticks(x)
  # ax.set_xticks
  # ax.set_xticklabels([v for v in test_hours[4:1800]])
  ax.tick_params(labelsize ='medium', pad = 5, direction = 'out', rotation = 45)
  ax.set_ylabel('Vehiculos')
  ax.set_xticks(np.arange(len(horas)))
  ax.legend()
  plt.xticks(rotation='vertical')
  plt.show()
  print("ATA -------> ", train_keys[i])
  print("-------------------- ACABA AQUI, CALLE NUEVA -----------------------")
print('Total train Score: %.4f RMSE' % (total_train_rmse))
print('Total train Score: %.4f MAE' % (total_train_mae))
print('Total train Score: %.4f MSE' % (total_train_mse))

print('Total test Score: %.4f RMSE' % (total_test_rmse))
print('Total test Score: %.4f MAE' % (total_test_mae))
print('Total test Score: %.4f MSE' % (total_test_mse))


print('AVG train Score: %.4f RMSE' % (total_train_rmse / features))
print('AVG train Score: %.4f MAE' % (total_train_mae / features))
print('AVG train Score: %.4f MSE' % (total_train_mse / features))


print('AVG test Score: %.4f RMSE' % (total_test_rmse / features))
print('AVG test Score: %.4f MAE' % (total_test_mae / features))
print('AVG test Score: %.4f MSE' % (total_test_mse / features))



In [ ]:
train_mse_score = mean_squared_error(y_train, train_predict)
print("TRAIN MSE", train_mse_score)
train_mae_score = mean_absolute_error(y_train, train_predict)
print("TRAIN MAE", train_mae_score)
train_rmse_score = math.sqrt(mean_squared_error(y_train, train_predict))
print("TRAIN RMSE", train_rmse_score)

val_mse_score = mean_squared_error(y_val, val_predict)
print("VAL MSE", val_mse_score)
val_mae_score = mean_absolute_error(y_val, val_predict)
print("VAL MAE", val_mae_score)
val_rmse_score = math.sqrt(mean_squared_error(y_val, val_predict))
print("VAL RMSE", val_rmse_score)

test_mse_score = mean_squared_error(y_test, test_predict)
print("TEST MSE", test_mse_score)
test_mae_score = mean_absolute_error(y_test, test_predict)
print("TEST MAE", test_mae_score)
test_rmse_score = math.sqrt(mean_squared_error(y_test, test_predict))
print("TEST RMSE", test_rmse_score)


Para el análisis del error

In [26]:
e_tipos = {}
e_barrios = {}
e_festivos = {}
e_calles = {}

for i in range(test_predict.shape[1]):
  ATA = test_keys[i]
  tipo = list(set(ordered_data.loc[ordered_data.ATA==ATA].tipo))
  barrio = list(set(ordered_data.loc[ordered_data.ATA==ATA].barrio))
  festivo = list(set(ordered_data.loc[ordered_data.ATA==ATA].festivo))
  test_score = math.sqrt(mean_squared_error(y_test[:, i], test_predict[:,  i]))
  test_mae_score = mean_absolute_error(y_test[:, i], test_predict[:,  i])
  test_mse_score = mean_squared_error(y_test[:, i], test_predict[:,  i])
  if len(tipo) > 0:
    e_calles[ATA] = test_mae_score
    tipo = tipo[0]
    if tipo == "resdiencial":
      tipo = "residencial"
    if tipo in e_tipos.keys():
      e_tipos[tipo] += test_mae_score
    else:
      e_tipos[tipo] = test_mae_score
  else:
    print(ATA)
  if len(barrio) > 0:
    barrio = barrio[0]
    if barrio in e_barrios.keys():
      e_barrios[barrio] += test_mae_score
    else:
      e_barrios[barrio] = test_mae_score
  if len(festivo) > 0:
    festivo = festivo[0]
    if festivo in e_festivos.keys():
      e_festivos[festivo] += test_mae_score
    else:
      e_festivos[festivo] = test_mae_score
    


A229
A302
B46
B47


In [27]:
e_tipos['terciaria']

4.647378

In [28]:
primarias = {}
secundarias = {}
terciarias = {}
entradas = {}
salidas = {}
residenciales = {}

for ATA in e_calles.keys():
  tipo = ordered_data[ordered_data.ATA==ATA].iloc[0].tipo
  if tipo == "primaria":
    primarias[ATA] = e_calles[ATA]
  elif tipo == "secundaria":
    secundarias[ATA] = e_calles[ATA]
  elif tipo == "terciaria":
    terciarias[ATA] = e_calles[ATA]
  elif tipo == "entrada":
    entradas[ATA] = e_calles[ATA]
  elif tipo == "salida":
    salidas[ATA] = e_calles[ATA]
  else:
    residenciales[ATA] = e_calles[ATA]        

  

In [30]:
primarias_ordenadas = sorted(primarias.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)
secundarias_ordenadas = sorted(secundarias.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)
terciarias_ordenadas = sorted(terciarias.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)
entradas_ordenadas = sorted(entradas.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)
salidas_ordenadas = sorted(salidas.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)
residenciales_ordenadas = sorted(residenciales.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)


In [ ]:
print("MAE medio calles primarias ", sum(primarias.values()) / len(primarias.values()))
print("MAE medio calles secundarias ", sum(secundarias.values()) / len(secundarias.values()))
print("MAE medio calles terciarias ", sum(terciarias.values()) / len(terciarias.values()))
print("MAE medio entradas ", sum(entradas.values()) / len(entradas.values()))
print("MAE medio salidas ", sum(salidas.values()) / len(salidas.values()))
print("MAE medio calles residenciales ", sum(residenciales.values()) / len(residenciales.values()))

In [ ]:
items = primarias_ordenadas[:10]
keys, values = [], []

for k, v in items:
  # keys.append(k)
  name = ordered_data[ordered_data.ATA==k].desc.iloc[0]
  if len(name) > 15:
    name = name[0:20] +"..."
  keys.append(name)
  values.append(v)
fig, ax = plt.subplots()
ax.set_title("10 calles primarias con más MAE")
ax.bar(keys, values)
ax.set_xlabel('Calle')
ax.set_ylabel('MAE')
plt.xticks(rotation=90)
mae_prim = 0
for v in primarias.values():
  mae_prim += v

plt.axhline(mae_prim/len(primarias.values()), 0, 1, label='media', color='red')
plt.show()

In [ ]:
items = secundarias_ordenadas[:10]
keys, values = [], []
for k, v in items:
  # keys.append(k)
  name = ordered_data[ordered_data.ATA==k].desc.iloc[0]
  if len(name) > 15:
    name = name[0:20] +"..."
  keys.append(name)
  values.append(v)
mae_secun = 0
for v in secundarias.values():
  mae_secun += v
fig, ax = plt.subplots()
ax.set_title("10 calles secundarias con más MAE")
ax.bar(keys, values)
ax.set_xlabel('Calle')
ax.set_ylabel('MAE')
plt.xticks(rotation = 90)
plt.axhline(mae_secun/len(secundarias.values()), 0, 1, label='media', color='red')
plt.show()

In [ ]:
items = terciarias_ordenadas[:10]
keys, values = [], []
for k, v in items:
  # keys.append(k)
  name = ordered_data[ordered_data.ATA==k].desc.iloc[0]
  if len(name) > 15:
    name = name[0:20] +"..."
  keys.append(name)
  values.append(v)
fig, ax = plt.subplots()
ax.set_title("10 calles terciarias con más MAE")
ax.bar(keys, values)
ax.set_xlabel('Calle')
plt.xticks(rotation = 90)
ax.set_ylabel('MAE')
mae_ter = 0
for v in terciarias.values():
  mae_ter += v

plt.axhline(mae_ter/len(terciarias.values()), 0, 1, label='media', color='red')
plt.show()

In [ ]:
items = entradas_ordenadas[:10]
keys, values = [], []
for k, v in items:
  # keys.append(k)
  name = ordered_data[ordered_data.ATA==k].desc.iloc[0]
  if len(name) > 15:
    name = name[0:20] +"..."
  keys.append(name)
  values.append(v)
fig, ax = plt.subplots()
ax.set_title("Entradas con más MAE")
ax.bar(keys, values)
ax.set_xlabel('Calle')
ax.set_ylabel('MAE')
plt.xticks(rotation = 90)
mae_ent = 0
for v in entradas.values():
  mae_ent += v

plt.axhline(mae_ent/len(entradas.values()), 0, 1, label='media', color='red')

plt.show()

In [ ]:
items = salidas_ordenadas[:10]
keys, values = [], []
for k, v in items:
  # keys.append(k)
  name = ordered_data[ordered_data.ATA==k].desc.iloc[0]
  if len(name) > 15:
    name = name[0:30] +"..."
  keys.append(name)
  values.append(v)
fig, ax = plt.subplots()
ax.set_title("Salidas con más MAE")
ax.bar(keys, values)
ax.set_xlabel('Calle')
ax.set_ylabel('MAE')
plt.xticks(rotation = 90)
mae_sal = 0
for v in salidas.values():
  mae_sal += v

plt.axhline(mae_sal/len(salidas.values()), 0, 1, label='media', color='red')

plt.show()

In [ ]:
items = residenciales_ordenadas[:10]
keys, values = [], []
for k, v in items:
  # keys.append(k)
  name = ordered_data[ordered_data.ATA==k].desc.iloc[0]
  if len(name) > 15:
    name = name[0:20] +"..."
  keys.append(name)
  values.append(v)
fig, ax = plt.subplots()
ax.set_title("10 calles residenciales con más MAE")
ax.bar(keys, values)
ax.set_xlabel('Calle')
ax.set_ylabel('MAE')
plt.xticks(rotation = 90)
mae_res = 0
for v in residenciales.values():
  mae_res += v

plt.axhline(mae_res/len(residenciales.values()), 0, 1, label='media', color='red')

plt.show()